# epub Creator - gravitytales.com

This notebook creates an epub file from a webnovel in http://gravitytales.com

In [ ]:
!pip install ebooklib requests-html pillow

In [ ]:
from ebooklib import epub
from requests_html import HTMLSession

from PIL import Image
import io

import sys

# Define the web-novel you want (Variables)

In [ ]:
book_title = 'King of Gods'
cover_image_url = ('https://vignette.wikia.nocookie.net/king-of-god/images/9/95/King-of-gods.jpg')
chapters_number = 854
url = 'http://gravitytales.com/novel/king-of-gods/kog-chapter-'
author = 'Fast Food Resturant ( 快餐店)'

# Download Chapters and create epub file

In [ ]:
book = epub.EpubBook()

# set metadata
book.set_identifier('')
book.set_title(book_title)
book.set_language('en')
#r_image = requests.get(cover_image_url)
#!wget https://vignette.wikia.nocookie.net/king-of-god/images/9/95/King-of-gods.jpg

with open('cover.jpg', 'wb') as handle:
        response = requests.get(cover_image_url, stream=True)

        if not response.ok:
            print(response)

        for block in response.iter_content(1024):
            if not block:
                break

            handle.write(block)


# add cover image
book.set_cover("image.jpg", open('cover.jpg', 'rb').read())

book.add_author(author)

# start session
session = HTMLSession()
r = session.get(url+str(chapters_number))

In [ ]:
toc_list = []
output_stream = sys.stdout
for chapter in range(1,chapters_number+1):
    r = session.get(url+str(chapter))
    if not r.html.find('h4', first=True): #chapter 847 has another url pattern (kog-chapter-847-1). This fixes the script.
        r = session.get(url+str(chapter)+'-1')
    content = r.html.find('#chapterContent', first=True)
    #print(content.html)
    chapter_title = r.html.find('h4', first=True)
    #print(chapter_title.text)
    epub_chapter = epub.EpubHtml(title=u'{}'.format(chapter_title.text), file_name='{}.xhtml'.format(chapter_title.text.replace(' ','_')), lang='hr')
    epub_chapter.content = content.html
    book.add_item(epub_chapter)
    toc_list.append(epub_chapter)
    
    sys.stdout.write("\rChapter %i" % chapter)
    sys.stdout.flush()
    
#print(toc_list)

In [ ]:
# define Table Of Contents
book.toc = tuple(toc_list)

# add default NCX and Nav file
book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

# define CSS style
style = 'BODY {color: white;}'
nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)

# add CSS file
book.add_item(nav_css)

# basic spine
book.spine = ['nav']+toc_list

# write to the file
epub.write_epub('{}.epub'.format(book_title.replace(' ', '_')), book, {})